##### Setup


In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
cd '/content/drive/My Drive/MLHC Project Resources/python_code/'

/content/drive/.shortcut-targets-by-id/107n_WmJFGDku3jDNPRYJskUDip3ElWv7/MLHC Project Resources/python_code


In [3]:
#@markdown Load packages 
!pip install pydicom
from load_model import *
from utils import *
import torchvision
import pandas as pd
import torch
import cv2
import os
import torch.nn.functional as F
import re

     |████████████████████████████████| 35.3MB 87kB/s 


In [0]:
#@markdown Load list of records 
record_df = pd.read_csv('./record_list_subset.csv')

In [0]:
#@markdown Helper function for getting the text of a report
#added it to Utils too but got a weird error there that I couldn't solve (?)
def get_report_text(study_id, overall_dataframe):
    """
    study_id: a unique identifier for a study done on a patient
              each study_id will have one report associated with it (report might relate to multiple images)
    overall_dataframe: the dataframe containing the report file paths
    """
    # Get the report
    record = overall_dataframe.loc[overall_dataframe.study_id == study_id]
    image_paths = record.path.iloc[0]
    study_id = str(study_id)

    report_path = re.sub(str('s' + study_id + '/.*$'), str('s' + study_id + '.txt'), str(image_paths))

    report_text = ''
    with open(report_path, 'r') as file:
        report_text = file.read().replace('\n', '')

    return report_text

#Example use: get_report_text(50414267, record_df)

In [0]:
#@markdown Create DB of reports
report_df=record_df.drop_duplicates(['subject_id','study_id'])[['subject_id','study_id']]
report_df["report_text"] = ""
for i in range(len(report_df.study_id)):
  s_id = report_df.iloc[i,1]
  report_df.iloc[i,2] = get_report_text(s_id, record_df)

##### Clinical BERT Semantic Similiarity

In [7]:
#@markdown Install package from https://github.com/AndriyMulyar/semantic-text-similarity
!pip install semantic-text-similarity 
from semantic_text_similarity.models import ClinicalBertSimilarity
from scipy.stats import pearsonr

#use GPU or CPU depedning on which is available
use_device = "cpu"
if torch.cuda.is_available():
  use_device = "cuda"
BERT = ClinicalBertSimilarity(device=use_device) 

#Example use:
#predictions = BERT.predict([("I thought LIME was a fruit.","Using LIME is a piece of cake.")])
#print(predictions)

     |████████████████████████████████| 419kB 2.7MB/s 
     |████████████████████████████████| 163kB 8.9MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 1.0MB 12.7MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
  Created wheel for python-levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144801 sha256=f3459afd56f5bf66253d305120254c09fea5b6764d35cf5bc52c03834d361604
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-levenshtein


100%|██████████| 401555686/401555686 [00:06<00:00, 57656450.33B/s]


In [0]:
#@markdown Function for getting the K-most similiar records given a study_id
def get_k_most_similiar_studies(study_id, k, report_df):
    """
    study_id: a unique identifier for a study done on a patient
    k: number of similiar studies
    report_df: df with all reports (and columns report_text and subject_id)
    """

    study = report_df.loc[report_df.study_id == study_id]
    study_text = study.report_text.iloc[0] #we will compare all report texts to this one

    similiarities = []
    for i in range(len(report_df.subject_id)):
      similiarity = BERT.predict([(study_text,report_df.iloc[i,2])]) #predict similarity with our BERT model
      similiarities.append(similiarity[0])

    similiarities = np.array(similiarities)
    top_k_index = similiarities.argsort()[-(k+1):][::-1] #index of k+1 elements with the highest value

    k_study_ids = []
    for i in range(len(report_df.subject_id)): #select corresponding study_ids
      if i in top_k_index and report_df.iloc[i,1] != study_id: k_study_ids.append(report_df.iloc[i,1])

    return k_study_ids


In [0]:
#Example use:
get_k_most_similiar_studies(53189527, 5, report_df)

[50771383, 56164612, 59988438]